## Natural Language Processing of Shakespeare Plays

#### Data Source: https://www.kaggle.com/kingburrito666/shakespeare-plays

#### References: 

* https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
* https://spacy.io/
* https://www.dataquest.io/blog/tutorial-text-classification-in-python-using-spacy/
* https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

Columns: Dataline, Play,PlayerLinenumber, ActSceneLine, Player, PlayerLine

* spaCy is an English language model
* spaCy is the best way to prepare your model for Deep Learning

Benefits of spaCy:

* Non-destructive tokenization
* Named entity recognition
* Support for 53+ languages
* 23 statistical models for 11 languages
* pretrained word vectors
* State-of-the-art speed
* Easy deep learning integration
* Part-of-speech tagging
* Labelled dependency parsing
* Syntax-driven sentence segmentation
* Built in visualizers for syntax and NER
* Convenient string-to-hash mapping
* Export to numpy data arrays
* Efficient binary serialization
* Easy model packaging and deployment
* Robust, rigorously evaluated accuracy

In [1]:
# Run in python console
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Wendy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

#### Install the modules

In [2]:
# pip install gensim

In [3]:
# pip install pyLDAvis

In [4]:
# pip install spacy

##### Run in terminal or command prompt (if not already satisified the requirement)

python -m spacy download en_core_web_sm

In [5]:
import spacy

In [6]:
# Load English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load("en_core_web_sm")

In [7]:
# Process whole documents
text = ("ACT I"
"SCENE I. London. The palace."
"Enter KING HENRY, LORD JOHN OF LANCASTER, the EARL of WESTMORELAND, SIR WALTER BLUNT, and others"
"So shaken as we are, so wan with care,"
"Find we a time for frighted peace to pant,"
"And breathe short-winded accents of new broils"
"To be commenced in strands afar remote."
"No more the thirsty entrance of this soil"
"Shall daub her lips with her own children's blood,"
"Nor more shall trenching war channel her fields,"
"Nor bruise her flowerets with the armed hoofs"
"Of hostile paces: those opposed eyes,"
"Which, like the meteors of a troubled heaven,"
"All of one nature, of one substance bred,"
"Did lately meet in the intestine shock"
"And furious close of civil butchery"
"Shall now, in mutual well-beseeming ranks,"
"March all one way and be no more opposed"
"Against acquaintance, kindred and allies:"
"The edge of war, like an ill-sheathed knife,"
"No more shall cut his master. Therefore, friends,"
"As far as to the sepulchre of Christ,"
"Whose soldier now, under whose blessed cross"
"We are impressed and engaged to fight,"
"Forthwith a power of English shall we levy,"
"Whose arms were moulded in their mothers' womb"
"To chase these pagans in those holy fields"
"Over whose acres walk'd those blessed feet"
"Which fourteen hundred years ago were nail'd"
"For our advantage on the bitter cross.")

In [8]:
doc = nlp(text)

In [9]:
# Analyze syntax

print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

Noun phrases: ['ACT ISCENE I. London', 'The palace', 'KING HENRY', 'LANCASTER', 'the EARL', 'WESTMORELAND', 'SIR WALTER BLUNT', 'we', 'so wan', 'care', 'we', 'frighted peace', 'pant', 'short-winded accents', 'strands', 'the thirsty entrance', 'this soilShall', 'her lips', "her own children's blood", 'war channel', 'her fields', 'her flowerets', 'the armed hoofsOf hostile paces', 'those opposed eyes', 'the meteors', 'a troubled heaven', 'one nature', 'one substance', 'civil butcheryShall', 'mutual well-beseeming ranks', 'no more opposedAgainst acquaintance', 'kindred', 'allies', 'The edge', 'war', 'an ill-sheathed knife', 'his master', 'the sepulchre', 'Christ', 'whose blessed crossWe', 'a power', 'English', 'we', 'Whose arms', "their mothers' wombTo chase", 'these pagans', 'those holy fieldsOver', 'our advantage', 'the bitter cross']
Verbs: ['act', 'enter', 'JOHN', 'shake', 'find', 'fright', 'breathe', 'commence', 'daub', 'trench', 'bruise', 'oppose', 'breed', 'meet', 'beseem', 'shall'

In [10]:
# Find named entities, phrases and concepts

for entity in doc.ents:
    print(entity.text, entity.label_)

ACT ISCENE I. London ORG
KING HENRY PERSON
the EARL of WESTMORELAND ORG
WALTER BLUNT PERSON
hoofsOf PRODUCT
one CARDINAL
one CARDINAL
shockAnd ORG
March DATE
one CARDINAL
kindred PERSON
Christ ORG
crossWe GPE
Forthwith PERSON
English LANGUAGE
fourteen hundred years ago DATE
the bitter cross ORG


In [11]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

## 1. Import the Shakespeare Plays dataset

In [12]:
# Import Dataset
df = pd.read_csv("Shakespeare_data.csv")
print(df.Play.unique())
df.head()

['Henry IV' 'Henry VI Part 1' 'Henry VI Part 2' 'Henry VI Part 3'
 'Alls well that ends well' 'As you like it' 'Antony and Cleopatra'
 'A Comedy of Errors' 'Coriolanus' 'Cymbeline' 'Hamlet' 'Henry V'
 'Henry VIII' 'King John' 'Julius Caesar' 'King Lear' 'Loves Labours Lost'
 'macbeth' 'Measure for measure' 'Merchant of Venice'
 'Merry Wives of Windsor' 'A Midsummer nights dream'
 'Much Ado about nothing' 'Othello' 'Pericles' 'Richard II' 'Richard III'
 'Romeo and Juliet' 'Taming of the Shrew' 'The Tempest' 'Timon of Athens'
 'Titus Andronicus' 'Troilus and Cressida' 'Twelfth Night'
 'Two Gentlemen of Verona' 'A Winters Tale']


,Dataline,Play,PlayerLinenumber,ActSceneLine,Player,PlayerLine
0,1,Henry IV,NaN,NaN,NaN,ACT I
1,2,Henry IV,NaN,NaN,NaN,SCENE I. London. The palace.
2,3,Henry IV,NaN,NaN,NaN,"Enter KING HENRY, LORD JOHN OF LANCASTER, the ..."
3,4,Henry IV,1.0,1.1.1,KING HENRY IV,"So shaken as we are, so wan with care,"
4,5,Henry IV,1.0,1.1.2,KING HENRY IV,"Find we a time for frighted peace to pant,"


##  2. Remove whitespace and other formatting

In [13]:
# Convert to list
data = df.PlayerLine.values.tolist()

pprint(data[:1])

['ACT I']


## 3.  Tokenize words and clean up words

#### Word Tokenization

* The process of breaking up words into text

In [14]:
# Word tokenization
from spacy.lang.en import English

In [15]:
# Load English tokenizer, tagger, parser, NER and word vectors
nlp = English()

In [16]:
text = """"ACT I"
"SCENE I. London. The palace."
"Enter KING HENRY, LORD JOHN OF LANCASTER, the EARL of WESTMORELAND, SIR WALTER BLUNT, and others"
"So shaken as we are, so wan with care,"
"Find we a time for frighted peace to pant,"
"And breathe short-winded accents of new broils"
"To be commenced in strands afar remote."
"No more the thirsty entrance of this soil"
"Shall daub her lips with her own children's blood,"
"Nor more shall trenching war channel her fields,"
"Nor bruise her flowerets with the armed hoofs"
"Of hostile paces: those opposed eyes,"
"Which, like the meteors of a troubled heaven,"
"All of one nature, of one substance bred,"
"Did lately meet in the intestine shock"
"And furious close of civil butchery"
"Shall now, in mutual well-beseeming ranks,"
"March all one way and be no more opposed"
"Against acquaintance, kindred and allies:"
"The edge of war, like an ill-sheathed knife,"
"No more shall cut his master. Therefore, friends,"
"As far as to the sepulchre of Christ,"
"Whose soldier now, under whose blessed cross"
"We are impressed and engaged to fight,"
"Forthwith a power of English shall we levy,"
"Whose arms were moulded in their mothers' womb"
"To chase these pagans in those holy fields"
"Over whose acres walk'd those blessed feet"
"Which fourteen hundred years ago were nail'd"
"For our advantage on the bitter cross."""

In [17]:
#  "nlp" Object is used to create documents with linguistic annotations.
my_doc = nlp(text)

In [18]:
# Create list of word tokens
token_list = []
for token in my_doc:
    token_list.append(token.text)
print(token_list)

['"', 'ACT', 'I', '"', '\n', '"', 'SCENE', 'I.', 'London', '.', 'The', 'palace', '.', '"', '\n', '"', 'Enter', 'KING', 'HENRY', ',', 'LORD', 'JOHN', 'OF', 'LANCASTER', ',', 'the', 'EARL', 'of', 'WESTMORELAND', ',', 'SIR', 'WALTER', 'BLUNT', ',', 'and', 'others', '"', '\n', '"', 'So', 'shaken', 'as', 'we', 'are', ',', 'so', 'wan', 'with', 'care', ',', '"', '\n', '"', 'Find', 'we', 'a', 'time', 'for', 'frighted', 'peace', 'to', 'pant', ',', '"', '\n', '"', 'And', 'breathe', 'short', '-', 'winded', 'accents', 'of', 'new', 'broils', '"', '\n', '"', 'To', 'be', 'commenced', 'in', 'strands', 'afar', 'remote', '.', '"', '\n', '"', 'No', 'more', 'the', 'thirsty', 'entrance', 'of', 'this', 'soil', '"', '\n', '"', 'Shall', 'daub', 'her', 'lips', 'with', 'her', 'own', 'children', "'s", 'blood', ',', '"', '\n', '"', 'Nor', 'more', 'shall', 'trenching', 'war', 'channel', 'her', 'fields', ',', '"', '\n', '"', 'Nor', 'bruise', 'her', 'flowerets', 'with', 'the', 'armed', 'hoofs', '"', '\n', '"', 'Of',

## Sentence tokenization  

##### Break text into sentences rather than words. This is called sentence tokenization

In [19]:
# Load English tokenizer, tagger, parser, NER and word vectors
nlp = English()

In [20]:
# Create the pipeline 'sentencizer' component
sbd = nlp.create_pipe('sentencizer')

In [21]:
# Add the component to the pipeline
nlp.add_pipe(sbd)

In [22]:
text = """ACT I"
"SCENE I. London. The palace."
"Enter KING HENRY, LORD JOHN OF LANCASTER, the EARL of WESTMORELAND, SIR WALTER BLUNT, and others"
"So shaken as we are, so wan with care,"
"Find we a time for frighted peace to pant,"
"And breathe short-winded accents of new broils"
"To be commenced in strands afar remote."
"No more the thirsty entrance of this soil"
"Shall daub her lips with her own children's blood,"
"Nor more shall trenching war channel her fields,"
"Nor bruise her flowerets with the armed hoofs"
"Of hostile paces: those opposed eyes,"
"Which, like the meteors of a troubled heaven,"
"All of one nature, of one substance bred,"
"Did lately meet in the intestine shock"
"And furious close of civil butchery"
"Shall now, in mutual well-beseeming ranks,"
"March all one way and be no more opposed"
"Against acquaintance, kindred and allies:"
"The edge of war, like an ill-sheathed knife,"
"No more shall cut his master. Therefore, friends,"
"As far as to the sepulchre of Christ,"
"Whose soldier now, under whose blessed cross"
"We are impressed and engaged to fight,"
"Forthwith a power of English shall we levy,"
"Whose arms were moulded in their mothers' womb"
"To chase these pagans in those holy fields"
"Over whose acres walk'd those blessed feet"
"Which fourteen hundred years ago were nail'd"
"For our advantage on the bitter cross."""

In [23]:
#  "nlp" Object is used to create documents with linguistic annotations

doc = nlp(text)

In [24]:
# create list of sentence tokens

sents_list = []
for sent in doc.sents:
    sents_list.append(sent.text)
print(sents_list)

['ACT I"\n"SCENE I. London.', 'The palace."', '\n"Enter KING HENRY, LORD JOHN OF LANCASTER, the EARL of WESTMORELAND, SIR WALTER BLUNT, and others"\n"So shaken as we are, so wan with care,"\n"Find we a time for frighted peace to pant,"\n"And breathe short-winded accents of new broils"\n"To be commenced in strands afar remote."', '\n"No more the thirsty entrance of this soil"\n"Shall daub her lips with her own children\'s blood,"\n"Nor more shall trenching war channel her fields,"\n"Nor bruise her flowerets with the armed hoofs"\n"Of hostile paces: those opposed eyes,"\n"Which, like the meteors of a troubled heaven,"\n"All of one nature, of one substance bred,"\n"Did lately meet in the intestine shock"\n"And furious close of civil butchery"\n"Shall now, in mutual well-beseeming ranks,"\n"March all one way and be no more opposed"\n"Against acquaintance, kindred and allies:"\n"The edge of war, like an ill-sheathed knife,"\n"No more shall cut his master.', 'Therefore, friends,"\n"As far as

## Cleaning Text Data: Removing Stopwords, creating a list of stopwords

In [25]:
# Stop words - noise does not assist with data analysis
# importing stop words from English language

import spacy
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

In [26]:
#Printing the total number of stop words:
print('Number of stop words: %d' % len(spacy_stopwords))

Number of stop words: 326


### Stopword list

In [27]:
# Printing first ten stop words:

print('First ten stop words: %s' % list(spacy_stopwords)[:20])

First ten stop words: ['back', 'yet', 'amongst', 'why', 'former', 'own', 'namely', 'nine', 'not', 'does', 'beside', 'becoming', 'beforehand', 'then', 'very', 'part', 'through', 'amount', 'although', 'bottom']


## Remove the stop words from the dataset

In [28]:
#Implementation of stop words:

filtered_sent=[]

In [29]:
#  "nlp" Object is used to create documents with linguistic annotations.
doc = nlp(text)

In [30]:
# filtering stop words
for word in doc:
    if word.is_stop==False:
        filtered_sent.append(word)
print("Filtered Sentence:",filtered_sent)

Filtered Sentence: [ACT, ", 
, ", SCENE, I., London, ., palace, ., ", 
, ", Enter, KING, HENRY, ,, LORD, JOHN, LANCASTER, ,, EARL, WESTMORELAND, ,, SIR, WALTER, BLUNT, ,, ", 
, ", shaken, ,, wan, care, ,, ", 
, ", Find, time, frighted, peace, pant, ,, ", 
, ", breathe, short, -, winded, accents, new, broils, ", 
, ", commenced, strands, afar, remote, ., ", 
, ", thirsty, entrance, soil, ", 
, ", Shall, daub, lips, children, blood, ,, ", 
, ", shall, trenching, war, channel, fields, ,, ", 
, ", bruise, flowerets, armed, hoofs, ", 
, ", hostile, paces, :, opposed, eyes, ,, ", 
, ", ,, like, meteors, troubled, heaven, ,, ", 
, ", nature, ,, substance, bred, ,, ", 
, ", lately, meet, intestine, shock, ", 
, ", furious, close, civil, butchery, ", 
, ", Shall, ,, mutual, -, beseeming, ranks, ,, ", 
, ", March, way, opposed, ", 
, ", acquaintance, ,, kindred, allies, :, ", 
, ", edge, war, ,, like, ill, -, sheathed, knife, ,, ", 
, ", shall, cut, master, ., ,, friends, ,, ", 
, ", far, sepulc

## Lexicon Normalization - converts high dimensional to low dimensional features

#### Lemmatization, a way of processing words that reduces them to their roots. E.g sun, sunny, sunshine
#### Stemming,  removes pre-fixes

In [31]:
# Implementing lemmatization
lem = nlp("ACT I"
"SCENE I. London. The palace."
"Enter KING HENRY, LORD JOHN OF LANCASTER, the EARL of WESTMORELAND, SIR WALTER BLUNT, and others"
"So shaken as we are, so wan with care,"
"Find we a time for frighted peace to pant,"
"And breathe short-winded accents of new broils"
"To be commenced in strands afar remote."
"No more the thirsty entrance of this soil"
"Shall daub her lips with her own children's blood,"
"Nor more shall trenching war channel her fields,"
"Nor bruise her flowerets with the armed hoofs"
"Of hostile paces: those opposed eyes,"
"Which, like the meteors of a troubled heaven,"
"All of one nature, of one substance bred,"
"Did lately meet in the intestine shock"
"And furious close of civil butchery"
"Shall now, in mutual well-beseeming ranks,"
"March all one way and be no more opposed"
"Against acquaintance, kindred and allies:"
"The edge of war, like an ill-sheathed knife,"
"No more shall cut his master. Therefore, friends,"
"As far as to the sepulchre of Christ,"
"Whose soldier now, under whose blessed cross"
"We are impressed and engaged to fight,"
"Forthwith a power of English shall we levy,"
"Whose arms were moulded in their mothers' womb"
"To chase these pagans in those holy fields"
"Over whose acres walk'd those blessed feet"
"Which fourteen hundred years ago were nail'd"
"For our advantage on the bitter cross.")
# finding lemma for each word
for word in lem:
    print(word.text,word.lemma_)

ACT ACT
ISCENE ISCENE
I. I.
London London
. .
The The
palace palace
. .
Enter Enter
KING KING
HENRY HENRY
, ,
LORD LORD
JOHN JOHN
OF OF
LANCASTER LANCASTER
, ,
the the
EARL EARL
of of
WESTMORELAND WESTMORELAND
, ,
SIR SIR
WALTER WALTER
BLUNT BLUNT
, ,
and and
othersSo othersSo
shaken shaken
as as
we we
are are
, ,
so so
wan wan
with with
care care
, ,
Find Find
we we
a a
time time
for for
frighted frighted
peace peace
to to
pant pant
, ,
And And
breathe breathe
short short
- -
winded winded
accents accents
of of
new new
broilsTo broilsTo
be be
commenced commenced
in in
strands strands
afar afar
remote remote
. .
No No
more more
the the
thirsty thirsty
entrance entrance
of of
this this
soilShall soilShall
daub daub
her her
lips lips
with with
her her
own own
children children
's 's
blood blood
, ,
Nor Nor
more more
shall shall
trenching trenching
war war
channel channel
her her
fields fields
, ,
Nor Nor
bruise bruise
her her
flowerets flowerets
with with
the the
armed armed
hoofsOf hoof

## Parts of Speech (PoS) Tagging 

* Identifying and tagging each word’s part of speech in the context of a sentence. Such as identifying a verb or a noun

In [32]:
# POS tagging

# importing the model en_core_web_sm of English for vocabluary, syntax & entities
import en_core_web_sm

In [33]:
# load en_core_web_sm of English for vocabluary, syntax & entities
nlp = en_core_web_sm.load()

In [34]:
#  "nlp" Objectis used to create documents with linguistic annotations.
docs = nlp(u"But this our purpose now is twelve month old,"
"And bootless 'tis to tell you we will go:"
"Therefore we meet not now. Then let me hear"
"Of you, my gentle cousin Westmoreland,"
"What yesternight our council did decree"
"In forwarding this dear expedience."
"My liege, this haste was hot in question,"
"And many limits of the charge set down"
"But yesternight: when all athwart there came"
"A post from Wales loaden with heavy news,"
"Whose worst was, that the noble Mortimer,"
"Leading the men of Herefordshire to fight"
"Against the irregular and wild Glendower,"
"Was by the rude hands of that Welshman taken,"
"A thousand of his people butchered,"
"Upon whose dead corpse there was such misuse,"
"Such beastly shameless transformation,"
"By those Welshwomen done as may not be"
"Without much shame retold or spoken of."
"It seems then that the tidings of this broil"
"Brake off our business for the Holy Land."
"This match'd with other did, my gracious lord,"
"For more uneven and unwelcome news"
"Came from the north and thus it did import:"
"On Holy-rood day, the gallant Hotspur there,"
"Young Harry Percy and brave Archibald,"
"That ever-valiant and approved Scot,"
"At Holmedon met,"
"Where they did spend a sad and bloody hour,"
"As by discharge of their artillery,"
"And shape of likelihood, the news was told,"
"For he that brought them, in the very heat"
"And pride of their contention did take horse,"
"Uncertain of the issue any way."
"Here is a dear, a true industrious friend,"
"Sir Walter Blunt, new lighted from his horse."
"Stain'd with the variation of each soil"
"Betwixt that Holmedon and this seat of ours,"
"And he hath brought us smooth and welcome news."
"The Earl of Douglas is discomfited:"
"Ten thousand bold Scots, two and twenty knights,"
"Balk'd in their own blood did Sir Walter see"
"On Holmedon's plains. Of prisoners, Hotspur took"
"Mordake the Earl of Fife, and eldest son"
"To beaten Douglas, and the Earl of Athol,"
"Of Murray, Angus, and Menteith:"
"And is not this an honourable spoil?"
"A gallant prize? ha, cousin, is it not?"
"In faith,"
"It is a conquest for a prince to boast of."
"Yea, there thou makest me sad and makest me sin"
"In envy that my Lord Northumberland"
"Should be the father to so blest a son,"
"A son who is the theme of honour's tongue,"
"Amongst a grove, the very straightest plant,"
"Who is sweet Fortune's minion and her pride:"
"Whilst I, by looking on the praise of him,"
"See riot and dishonour stain the brow"
"Of my young Harry. O that it could be proved"
"That some night-tripping fairy had exchanged"
"In cradle-clothes our children where they lay,"
"And call'd mine Percy, his Plantagenet!"
"Then would I have his Harry, and he mine."
"But let him from my thoughts. What think you, coz,"
"Of this young Percy's pride? the prisoners,"
"Which he in this adventure hath surprised,"
"To his own use he keeps, and sends me word,"
"I shall have none but Mordake Earl of Fife."
"This is his uncle's teaching, this is Worcester,"
"Malevolent to you in all aspects,"
"Which makes him prune himself, and bristle up"
"The crest of youth against your dignity."
"But I have sent for him to answer this,"
"And for this cause awhile we must neglect"
"Our holy purpose to Jerusalem."
"Cousin, on Wednesday next our council we"
"Will hold at Windsor, so inform the lords:"
"But come yourself with speed to us again,"
"For more is to be said and to be done"
"Than out of anger can be uttered."
"I will, my liege.")

for word in docs:
    print(word.text,word.pos_)

But CCONJ
this DET
our DET
purpose NOUN
now ADV
is AUX
twelve NUM
month NOUN
old ADJ
, PUNCT
And CCONJ
bootless NOUN
' PUNCT
tis VERB
to PART
tell VERB
you PRON
we PRON
will VERB
go VERB
: PUNCT
Therefore ADV
we PRON
meet VERB
not PART
now ADV
. PUNCT
Then ADV
let VERB
me PRON
hearOf NOUN
you PRON
, PUNCT
my DET
gentle ADJ
cousin NOUN
Westmoreland PROPN
, PUNCT
What PRON
yesternight VERB
our DET
council NOUN
did AUX
decreeIn PROPN
forwarding VERB
this DET
dear ADJ
expedience NOUN
. PUNCT
My DET
liege NOUN
, PUNCT
this DET
haste NOUN
was AUX
hot ADJ
in ADP
question NOUN
, PUNCT
And CCONJ
many ADJ
limits NOUN
of ADP
the DET
charge NOUN
set VERB
downBut PROPN
yesternight NOUN
: PUNCT
when ADV
all DET
athwart VERB
there ADV
cameA PROPN
post NOUN
from ADP
Wales PROPN
loaden ADJ
with ADP
heavy ADJ
news NOUN
, PUNCT
Whose DET
worst ADJ
was AUX
, PUNCT
that SCONJ
the DET
noble ADJ
Mortimer PROPN
, PUNCT
Leading VERB
the DET
men NOUN
of ADP
Herefordshire PROPN
to PART
fightAgainst VERB
the DET


## Entity Detection

In [35]:
#for visualization of Entity detection importing displacy from spacy:

from spacy import displacy

shakespeare= nlp(u""""But this our purpose now is twelve month old,"
"And bootless 'tis to tell you we will go:"
"Therefore we meet not now. Then let me hear"
"Of you, my gentle cousin Westmoreland,"
"What yesternight our council did decree"
"In forwarding this dear expedience."
"My liege, this haste was hot in question,"
"And many limits of the charge set down"
"But yesternight: when all athwart there came"
"A post from Wales loaden with heavy news,"
"Whose worst was, that the noble Mortimer,"
"Leading the men of Herefordshire to fight"
"Against the irregular and wild Glendower,"
"Was by the rude hands of that Welshman taken,"
"A thousand of his people butchered,"
"Upon whose dead corpse there was such misuse,"
"Such beastly shameless transformation,"
"By those Welshwomen done as may not be"
"Without much shame retold or spoken of."
"It seems then that the tidings of this broil"
"Brake off our business for the Holy Land."
"This match'd with other did, my gracious lord,"
"For more uneven and unwelcome news"
"Came from the north and thus it did import:"
"On Holy-rood day, the gallant Hotspur there,"
"Young Harry Percy and brave Archibald,"
"That ever-valiant and approved Scot,"
"At Holmedon met,"
"Where they did spend a sad and bloody hour,"
"As by discharge of their artillery,"
"And shape of likelihood, the news was told,"
"For he that brought them, in the very heat"
"And pride of their contention did take horse,"
"Uncertain of the issue any way."
"Here is a dear, a true industrious friend,"
"Sir Walter Blunt, new lighted from his horse."
"Stain'd with the variation of each soil"
"Betwixt that Holmedon and this seat of ours,"
"And he hath brought us smooth and welcome news."
"The Earl of Douglas is discomfited:"
"Ten thousand bold Scots, two and twenty knights,"
"Balk'd in their own blood did Sir Walter see"
"On Holmedon's plains. Of prisoners, Hotspur took"
"Mordake the Earl of Fife, and eldest son"
"To beaten Douglas, and the Earl of Athol,"
"Of Murray, Angus, and Menteith:"
"And is not this an honourable spoil?"
"A gallant prize? ha, cousin, is it not?"
"In faith,"
"It is a conquest for a prince to boast of."
"Yea, there thou makest me sad and makest me sin"
"In envy that my Lord Northumberland"
"Should be the father to so blest a son,"
"A son who is the theme of honour's tongue,"
"Amongst a grove, the very straightest plant,"
"Who is sweet Fortune's minion and her pride:"
"Whilst I, by looking on the praise of him,"
"See riot and dishonour stain the brow"
"Of my young Harry. O that it could be proved"
"That some night-tripping fairy had exchanged"
"In cradle-clothes our children where they lay,"
"And call'd mine Percy, his Plantagenet!"
"Then would I have his Harry, and he mine."
"But let him from my thoughts. What think you, coz,"
"Of this young Percy's pride? the prisoners,"
"Which he in this adventure hath surprised,"
"To his own use he keeps, and sends me word,"
"I shall have none but Mordake Earl of Fife."
"This is his uncle's teaching, this is Worcester,"
"Malevolent to you in all aspects,"
"Which makes him prune himself, and bristle up"
"The crest of youth against your dignity."
"But I have sent for him to answer this,"
"And for this cause awhile we must neglect"
"Our holy purpose to Jerusalem."
"Cousin, on Wednesday next our council we"
"Will hold at Windsor, so inform the lords:"
"But come yourself with speed to us again,"
"For more is to be said and to be done"
"Than out of anger can be uttered."
"I will, my liege.""")

entities=[(i, i.label_, i.label) for i in shakespeare.ents]
entities

[(twelve month old, 'DATE', 391),
 (Westmoreland, 'PERSON', 380),
 (Wales, 'GPE', 384),
 (Mortimer, 'PERSON', 380),
 (Herefordshire, 'GPE', 384),
 (Welshman, 'ORG', 383),
 (A thousand, 'CARDINAL', 397),
 (the Holy Land, 'ORG', 383),
 (On Holy-rood day, 'WORK_OF_ART', 388),
 ("Young Harry Percy and, 'WORK_OF_ART', 388),
 (Archibald, 'PERSON', 380),
 (Scot, 'PERSON', 380),
 (Holmedon, 'GPE', 384),
 (hour, 'TIME', 392),
 (Walter Blunt, 'PERSON', 380),
 (Betwixt, 'NORP', 381),
 (Holmedon, 'GPE', 384),
 (The Earl of Douglas, 'WORK_OF_ART', 388),
 (Ten thousand, 'CARDINAL', 397),
 (Scots, 'ORG', 383),
 (two, 'CARDINAL', 397),
 (twenty, 'CARDINAL', 397),
 (Walter, 'PERSON', 380),
 (Holmedon, 'GPE', 384),
 (Mordake the Earl of Fife, 'WORK_OF_ART', 388),
 (Douglas, 'PERSON', 380),
 (the Earl of Athol, 'WORK_OF_ART', 388),
 (Murray, Angus, 'PERSON', 380),
 (Yea, 'PERSON', 380),
 (Northumberland, 'PERSON', 380),
 (Amongst, 'WORK_OF_ART', 388),
 (Fortune, 'ORG', 383),
 (Whilst I, by looking on the

In [37]:
# Visualise the input text with displacy

displacy.render(shakespeare, style = "ent",jupyter = True)

## Dependency Parsing

* Determine the meaning of a sentence by analyzing how it’s constructed to determine how the individual words relate to each other.
* Displacy helps with the construction

In [38]:
docp = nlp ("But this our purpose now is twelve month old,"
"And bootless 'tis to tell you we will go:"
"Therefore we meet not now. Then let me hear"
"Of you, my gentle cousin Westmoreland,"
"What yesternight our council did decree"
"In forwarding this dear expedience.")

for chunk in docp.noun_chunks:
   print(chunk.text, chunk.root.text, chunk.root.dep_,
          chunk.root.head.text)

this our purpose purpose nsubj is
you you dobj tell
we we nsubj go
we we nsubj meet
me me dobj let
my gentle cousin cousin appos you
Westmoreland Westmoreland appos cousin
our council council nsubj did
decreeIn decreeIn nsubj forwarding
this dear expedience expedience dobj forwarding


In [39]:
# displacy visualizer

displacy.render(docp, style="dep", jupyter= True)

In [40]:
# Example 2

docp = nlp (" The NSW bushfires devasted a lot homes last week, there will be a long rebuilding of lives.")

for chunk in docp.noun_chunks:
   print(chunk.text, chunk.root.text, chunk.root.dep_,
          chunk.root.head.text)

 The NSW bushfires bushfires nsubj devasted
a lot homes homes dobj devasted
a long rebuilding rebuilding attr be
lives lives pobj of


In [41]:
# displacy visualizer

displacy.render(docp, style="dep", jupyter= True)

## Word Vector Representation

* A word vector is a numeric representation of a word that communicates its relationship to other words.Looking at the proximity of similar words using coordinates

* E.g the name Percy

In [42]:
import en_core_web_sm
nlp = en_core_web_sm.load()
percy = nlp(u'percy')
print(percy.vector.shape)
print(percy.vector)

(96,)
[ 3.2803802  -2.9817076   1.0940012   0.06815997  3.6642241   0.7598362
  1.4744527   1.6251982   1.0179482   1.1121266   3.2019553   0.10745922
  3.82013    -0.99108934  1.1341643  -0.11255711  0.58071095  2.3828812
 -1.2474027  -1.8118596   0.65091825  0.5728332   0.79027236 -0.9660708
 -0.22417854 -0.41104496  0.20417607 -3.5825176   2.8488975  -1.7978574
  1.075335   -0.95171833  0.00948098  2.201458    3.7402136  -0.31927556
  1.6363927  -0.7086991  -4.1015754  -0.8468743   1.8261789   1.6956387
 -0.19908196 -4.2831955   0.57832915 -1.1681477  -1.1425133  -0.6977829
 -1.1665349   0.26478255  1.4251769  -0.7935762  -0.24110645 -2.7856297
 -0.42935884 -1.9319078   0.9054253   3.1108668   1.409328    1.2341056
  0.30979902  0.6174552  -2.0923703  -3.5340824   1.0692596  -3.8384485
 -0.2615428  -4.7087126   2.9007015  -1.4799047  -1.8345451   0.7603681
  3.172385    1.5798618  -2.7121332  -0.06233608  1.8596227  -0.69003415
 -0.82352567  4.4042153   1.0887163  -1.2084568  -0.327